- This notebook is for testing whether the notebook axelth_849_modeling.ipynb can be running on Google Colab.

- First, I downloaded the latest hasp github code from the URL below to my local machine.
https://github.com/axelth/hasp


- Then I uploaded them to My Google Drive: MyDrive/Colab Notebooks/.


- I opened this notebook from my Google Drive: /MyDrive/Colab Noteboks/hasp/notebooks/.


- I also uploaded converted UrbanSound8k data to MyDrive/Data/hasp_test_data/.


- Additional code for running this notebook on Google Colab are with comment.


This notebook is to contain notes on different models, leading up to a decision on which model to use in production.

This notebook incorporates the dataloader defined in https://github.com/axelth/us8kdata , and will also incorporate the feature extraction pipeline prototyped in notebooks/.... and implemented in hasp/{features.py, pipeline.py} etc.

imports for individual models are performed in their respective section to make it easier to see which code to transfer to the library after model selection

### Additional code
- To install the packages for us8kdata
- This is needed for running this notebook on Google Colab.

In [1]:
pip install git+https://github.com/axelth/us8kdata.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/axelth/us8kdata.git to /tmp/pip-req-build-xvt7tljr
  Running command git clone -q https://github.com/axelth/us8kdata.git /tmp/pip-req-build-xvt7tljr


In [2]:
!pwd

/content


In [3]:
import os
print(f"Current Working Directory is {os.getcwd()}")

Current Working Directory is /content


### Additional Code
- Mount the Google Driver to Google Colab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls drive/MyDrive/Data/hasp_test_data/data

data   fold10  fold3  fold5  fold7  fold9
fold1  fold2   fold4  fold6  fold8  metadata


In [6]:
!ls drive/MyDrive/Colab\ Notebooks/hasp/notebooks

axelth_849_dataset_analysis.ipynb
axelth_849_modeling.ipynb
axelth_849_preproc_pipeline_for_mean_mfcc_feature.ipynb
carlq89_849_ag_xgb.ipynb
carlq89_849_knn.ipynb
carlq89_849_learn_curves.ipynb
carlq89_849_preproc_pipe.ipynb
carlq89_849_random_forest.ipynb
carlq89_849_test_pipeline.ipynb
carlq89_849_xgb.ipynb
shige-github_849_augmentation.ipynb
shige-github_849_dataset_analysis_ESC-50.ipynb
shige-github_849_GoogleColab_modeling.ipynb


In [7]:
GoogleDrive_hasp_notebooks_dir = "/content/drive/MyDrive/Colab Notebooks/hasp/notebooks/"
GoogleDrive_hasp_dir = "/content/drive/MyDrive/Colab Notebooks/hasp/"
GoogleDrive_audio_data = "/content/drive/MyDrive/Data/hasp_test_data/data/"

import os
print(f"Current Working Directory is {os.getcwd()}")
os.chdir(GoogleDrive_hasp_dir)
print(f"Current Working Directory has changed to {os.getcwd()}")

Current Working Directory is /content
Current Working Directory has changed to /content/drive/MyDrive/Colab Notebooks/hasp


In [8]:
%load_ext autoreload
%autoreload 

In [9]:
pip install audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.metrics import classification_report, confusion_matrix

from us8kdata.loader import UrbanSound8K
from hasp.feature_pipeline import make_feature_pipeline
from hasp.feature_pipeline import make_oversampled_feature_pipeline
from hasp.augmenting_oversampler import AugmentingRandomOversampler as OverSampler
from hasp.util import combine_classes

In [11]:
os.path.dirname(GoogleDrive_audio_data)

'/content/drive/MyDrive/Data/hasp_test_data/data'

In [12]:
!ls -l /content/drive/MyDrive/Data/hasp_test_data/data/metadata/

total 594
-rw------- 1 root root 303653 Aug 19 12:39 urbansound8K.csv
-rw------- 1 root root 303653 Aug 19 12:38 UrbanSound8K.csv


In [13]:
!cp /content/drive/MyDrive/Data/hasp_test_data/data/metadata/UrbanSound8K.csv /content/drive/MyDrive/Data/hasp_test_data/data/metadata/urbansound8K.csv

In [14]:
!ls -l /content/drive/MyDrive/Data/hasp_test_data/data/metadata/

total 594
-rw------- 1 root root 303653 Aug 19 12:51 urbansound8K.csv
-rw------- 1 root root 303653 Aug 19 12:38 UrbanSound8K.csv


In [ ]:
# data = UrbanSound8K('../hasp/data'
data = UrbanSound8K(os.path.dirname(GoogleDrive_audio_data))

train_fold = [samples for samples in data.fold_audio_generator(fold=[1, 2, 3, 6, 7, 8, 9, 10])]
val_fold = [samples for samples in data.fold_audio_generator(fold=[4])]
test_fold = [samples for samples in data.fold_audio_generator(fold=[5])]
train_y = data.filter_metadata(fold=[1, 2, 3, 6, 7, 8, 9, 10]).classID
val_y = data.filter_metadata(fold=[4]).classID
test_y = data.filter_metadata(fold=[5]).classID

In [ ]:
data.metadata[['classID', 'class']]

In [ ]:
feature_pipe = make_feature_pipeline()
feature_pipe

In [ ]:
train_pre = feature_pipe.fit_transform(train_fold)
val_pre = feature_pipe.transform(val_fold)
#test_pre = feature_pipe.transform(test_fold)

# Single sample classification
Model architectures for treating each audio file as a single example and classifying it in one go.

## Logistic Regression (baseline)

### LASSO

## KNN

## SVC

In [ ]:
from sklearn.svm import SVC
model_svc = SVC(C=4)
model_svc.fit(train_pre, train_y)

In [ ]:
#val_pre.shape
svc_val_prediction = model_svc.predict(val_pre)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], svc_val_prediction)
))
print(classification_report(
     val_y, 
    svc_val_prediction))
pd.DataFrame(confusion_matrix(val_y, svc_val_prediction), 
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

## RandomForest

## XGBoost

In [ ]:
from xgboost import XGBClassifier
#model_xgb = XGBClassifier()
#model_xgb.fit(train_pre, train_y)

In [ ]:
xgb_val_prediction = model_xgb.predict(val_pre)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_val_prediction)
))
print(classification_report(
     val_y, 
    xgb_val_prediction))
pd.DataFrame(confusion_matrix(val_y, xgb_val_prediction),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

### XGBoost with oversampled features


In [ ]:
oversampled_pipe = make_oversampled_feature_pipeline()
oversampled_pipe

In [ ]:
oversampled_pipe.get_params()

In [ ]:
oversampled_pipe.set_params(estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),                          
                           mean_mfcc__kw_args={'fmin':300, 'fmax':4000},
                           over_sampler=OverSampler())

In [ ]:
%%time
oversampled_pipe.fit(train_fold, train_y);

In [ ]:
xgb_ovsamp_val_pred = oversampled_pipe.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_ovsamp_val_pred)
))
print(classification_report(
     val_y, 
    xgb_ovsamp_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_ovsamp_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

#### oversampled gridsearch

In [ ]:
mean_mfcc_grid = []
for n_mfcc in [13, 16, 20]:#range(13, 26):
    for n_fft in range(256, 1025, 256):
        for fmin in [0.0, 300.0, 500.0]: #50.0, 100.0, 200.0, 300.0, 400.0, 500.0]:
            for fmax in [None, 3000.0, 4000.0, 5000.0]: #, 4000.0, 3000.0]:
                mean_mfcc_grid.append({'sr':16000, 'n_mfcc': n_mfcc, 'n_fft': n_fft,
                                       'hop_length': 128, 'fmin': fmin, 'fmax': fmax})
len(mean_mfcc_grid)

In [ ]:
xgb_grid = [
    XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=8, objective='multi:softmax'),
    XGBClassifier(n_estimators=400, learning_rate=0.025, max_depth=8, objective='multi:softmax')
]

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
gs = GridSearchCV(oversampled_pipe, 
                  dict(mean_mfcc__kw_args=mean_mfcc_grid,
                      estimator=xgb_grid), 
                  cv=2, verbose=2, n_jobs=1)
gs_res = gs.fit(train_fold, train_y)

In [ ]:
gs_res.best_params_

In [ ]:
xgb_gs_val_pred = gs.best_estimator_.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_gs_val_pred)
))
print(classification_report(
     val_y, 
    xgb_gs_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_gs_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

In [ ]:
%%time
post_search_pipeline = make_oversampled_feature_pipeline()
post_search_pipeline.set_params(
**{'estimator': XGBClassifier(learning_rate=0.05, max_depth=8,
               n_estimators=400, objective='multi:softmax'),
 'mean_mfcc__kw_args': {'sr': 16000,'n_mfcc': 16,
  'n_fft': 256,'hop_length': 128,
  'fmin': 0.0, 'fmax': 3000.0, 'n_mels': 98},
  'over_sampler': OverSampler(augment_method='AddGaussianNoise')}
)
post_search_pipeline.fit(train_fold, train_y)

In [ ]:
xgb_ps_val_pred = post_search_pipeline.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_ps_val_pred)
))
print(classification_report(
     val_y, 
    xgb_ps_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_ps_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

## VGGish

# Sequence of samples classification
Model architectures for treating each file as a sequence of sample frames, making the classification either based on a subset or on the frame as a whole.

## HMM

## RNN